<a href="https://colab.research.google.com/github/bhatsbharath/generative_ai_agents/blob/main/multi_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web searching
Tool used to fetch data from the web

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize the tool with custom settings
tool = DuckDuckGoSearchRun(
    description="A custom DuckDuckGo search tool for finding programming resources.",
    verbose=True
)

# Example usage
result = tool.invoke("LangChain framework")

word_count = len(result.split())
print("\nNumber of words:", word_count)
print(result)

# Multi Agent Researcher tool

Multi Agent Researcher tool using a researcher and superviser agent

In [ ]:
import re
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor


def clean_text(text: str):
    cleaned_text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
    return cleaned_text.strip()

model = ChatOllama(model="llama3.2")

Defines a function clean_text that removes content enclosed in <think> tags from a given text using regex.

Initializes a ChatOllama model instance named model.

Sets up three distinct prompt strings for different agents in a workflow:

Query Refiner Agent: Refines user queries to be more specific and actionable.

Research Agent: Handles research-related tasks and provides summarized results.

Supervisor Agent: Oversees the interaction between the refiner and researcher, coordinating the workflow and producing the final response.

In [ ]:
def clean_text(text: str):
    cleaned_text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
    return cleaned_text.strip()

model = ChatOllama(model="llama3.2")

query_refiner_prompt = (
    "You are a query refiner agent.\n\n"
    "INSTRUCTIONS:\n"
    "- Refine the user's query to make it more specific and actionable\n"
    "- Respond ONLY with the refined query, do NOT include any other text."
)
research_prompt = (
    "You are a research agent.\n\n"
    "INSTRUCTIONS:\n"
    "- Assist ONLY with research-related tasks, DO NOT do anything else\n"
    "- After you're done with your research, respond to the supervisor directly\n"
    "- Respond ONLY with the summary of the results, do NOT include ANY other text."
)
supervisor_prompt = (
    "You are a supervisor agent.\n\n"
    "INSTRUCTIONS:\n"
    "- Manage the workflow between the query refiner and researcher agents\n"
    "- Use the Query Refiner Agent to correct typos and enhance the query for better search relevance\n"
    "- Use the Research Agent to retrieve up-to-date information from the web\n"
    "- Respond ONLY with the final response, do NOT include any other text."
)


Query Refiner Agent: Uses no external tools to refine user queries.

Research Agent: Equipped with DuckDuckGo search to perform web research.

Supervisor Agent: Coordinates the workflow between the Query Refiner and Research agents to produce the final output.

In [ ]:
query_refiner_agent = create_react_agent(
    model=model,
    tools=[],
    prompt=query_refiner_prompt,
    name="query_refiner_agent"
)
research_agent = create_react_agent(
    model=model,
    tools=[DuckDuckGoSearchRun()],
    prompt=research_prompt,
    name="research_agent"
)
supervisor_agent = create_supervisor(
    model=model,
    agents=[query_refiner_agent, research_agent],
    prompt=supervisor_prompt
)

Compiles the supervisor_agent into an application.

Sends the user query to the compiled app.

Prints the formatted messages from the app’s response.

In [ ]:
app = supervisor_agent.compile()
import streamlit as st

query = st.text_input("What is neuromorphic computing")

result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "What is neuromorphic computing"
        }
    ]
})

for message in result["messages"]:
    print(message.pretty_print())
    print()


In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))